In [7]:
import pandas as pd
import torch

In [12]:
train_data = pd.read_csv('./dataset/train.csv')
test_data = pd.read_csv('./dataset/test.csv')

labels = train_data['target']
train_features = train_data.drop(columns=['target'])
all_features = pd.concat([train_features, test_data], ignore_index=True)
all_features
all_features = all_features.drop(columns=['id', 'location'])

all_features['keyword'].fillna('unknown', inplace=True)
all_features = pd.get_dummies(all_features, columns=['keyword'], prefix='kw')

/var/folders/dh/7984j7ln2b9005qghdnnbqbc0000gn/T/ipykernel_90014/2511795653.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_features['keyword'].fillna('unknown', inplace=True)


In [13]:
all_features.head()

,text,kw_ablaze,kw_accident,kw_aftershock,kw_airplane%20accident,kw_ambulance,kw_annihilated,kw_annihilation,kw_apocalypse,kw_armageddon,...,kw_weapons,kw_whirlwind,kw_wild%20fires,kw_wildfire,kw_windstorm,kw_wounded,kw_wounds,kw_wreck,kw_wreckage,kw_wrecked
0,Our Deeds are the Reason of this #earthquake M...,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,Forest fire near La Ronge Sask. Canada,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,All residents asked to 'shelter in place' are ...,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,"13,000 people receive #wildfires evacuation or...",False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Just got sent this photo from Ruby #Alaska as ...,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [14]:
train_data['text'].str.len().max()

157

In [16]:
all_features.dtypes

text                      object
kw_ablaze                   bool
kw_accident                 bool
kw_aftershock               bool
kw_airplane%20accident      bool
                           ...  
kw_wounded                  bool
kw_wounds                   bool
kw_wreck                    bool
kw_wreckage                 bool
kw_wrecked                  bool
Length: 223, dtype: object

In [3]:
from dataset import TweetSet, load_and_process_data
from model import MultimodalModel

/Users/robinsonxing/miniconda3/envs/d2l/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/robinsonxing/miniconda3/envs/d2l/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [27]:
# 加载并处理数据
keyword_data, location_data, text_data, labels = load_and_process_data(mode='train')
dataset = TweetSet(pd.concat([keyword_data, location_data, text_data], axis=1), labels)

/Users/robinsonxing/Documents/Code Source/kaggle/NLP_with_Disaster_Tweets/dataset.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_data['keyword'].fillna('unknown', inplace=True)
/Users/robinsonxing/miniconda3/envs/d2l/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [28]:
keyword_features = dataset.data.filter(like='kw').values

In [25]:
for i, element in enumerate(keyword_features):
    if element.dtype != 'float64':
        print(f"元素 {i} 的类型: {element.dtype}")

In [29]:
keyword_features = torch.tensor(keyword_features, dtype=torch.float32)